# Get information about water consume of the mice
I used that for daily monitoring of the water consume of the mice. No real magic about it. We get the basic count of waterdrops for the night (or whatever time we adjust the query to) and with the second request we also get under which condition they got the water. This helps to see if they get drink while headfixed (because they just drink entry rewards or at no fix trials) or cannot reach the spout or freeze. When the "pass" condition appears it indicates that they drink after the RFID sensor lost track and is probably not at the best position. That is especially useful for early phases where we have to change the position of the RFID reader a few times while spout training. If you run the query now you will get nothing back, because it uses (Date(NOW()) but feel free to put whatever date into it that you want. e.g.

Date("2018-12-05")

In [3]:
import pandas as pd
import pymysql
from password import database_password as DBpwd
from password import database_user as DBuser
from password import database_host as DBhost
from password import database as DB

def generate_commands(condition):
    if condition == "no":
        query = """SELECT Date(`Timestamp`),`Mouse`, COUNT(*) FROM `rewards` WHERE `Timestamp` >= (Date("2018-12-05") - 2) 
                    Group BY Date(`Timestamp`),`Mouse`"""
    elif condition == "yes":
        query = """SELECT Date(`Timestamp`),`Mouse`,`Reward_type`, COUNT(*) FROM `rewards` WHERE `Timestamp` >= (Date(NOW()) - INTERVAL 8 HOUR)  
                            Group BY Date(`Timestamp`),`Mouse`,`Reward_type`"""
    return query

def getFromDatabase(query):
    db2 = pymysql.connect(host=DBhost, user=DBuser, db=DB, password=DBpwd)
    cur2 = db2.cursor()
    try:
        cur2.execute(query)
        rows = cur2.fetchall()
    except pymysql.Error as e:
        try:
            print("MySQL Error [%d]: %s" % (e.args[0], e.args[1]))
            return None
        except IndexError:
            print("MySQL Error: %s" % str(e))
            return None
    except TypeError as e:
        print("MySQL Error: TypeError: %s" % str(e))
        return None
    except ValueError as e:
        print("MySQL Error: ValueError: %s" % str(e))
        return None
    db2.close()
    return rows



data = list(getFromDatabase(query = generate_commands("yes")))
df = pd.DataFrame(data=data,columns=["Date","Mouse","condition","Waterdrops"])
print(df)
df.to_csv("water+.csv")
data = list(getFromDatabase(query = generate_commands("no")))
df = pd.DataFrame(data=data,columns=["Date","Mouse","Waterdrops"])
print(df)
df.to_csv("water-.csv")

Empty DataFrame
Columns: [Date, Mouse, condition, Waterdrops]
Index: []
          Date      Mouse  Waterdrops
0   2018-12-03  801010205          99
1   2018-12-03  801010219          71
2   2018-12-04  801010205          66
3   2018-12-04  801010219         164
4   2018-12-05  801010205          49
5   2018-12-05  801010219          53
6   2018-12-05  801010270          52
7   2018-12-06  801010205          67
8   2018-12-06  801010219          99
9   2018-12-06  801010270         101
10  2018-12-07  801010219          88
11  2018-12-07  801010270          51
12  2018-12-08  801010219          14
13  2018-12-09  801010219          65
14  2018-12-09  801010270          73
15  2018-12-10  801010219          67
16  2018-12-10  801010270          48
17  2018-12-11  801010219         128
18  2018-12-11  801010270          74
19  2018-12-12  801010219         112
20  2018-12-13  801010219          95
21  2018-12-13  801010270         105
22  2018-12-14  801010219          64
23  2018-12-14  